<a href="https://colab.research.google.com/github/eerisken/deep-learning-experiments/blob/main/LTN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules
from sklearn.preprocessing import MultiLabelBinarizer

drive.mount('/content/drive')

# Load the data
df = pd.read_csv('/content/drive/My Drive/movie-genre-prediction/production.csv')

# Association rule mining
transactions = df['Output'].str.split(', ').tolist()
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = fpgrowth(df_encoded, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
high_confidence_rules = rules[(rules['confidence'] > 0.3) & (rules['support'] > 0.01)]

# Data preprocessing for multi-label classification
df['description'] = df['Input'].apply(lambda x: x.split('\n\n', 1)[1] if '\n\n' in x else '')
df['Output-Label'] = df['Output'].str.split(', ')
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['Output-Label'])

# Display results
display(df.head())
display(high_confidence_rules)
print("Descriptions:")
display(df['description'].head())
print("\nBinary Labels (y):")
display(y[:5])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,user_interaction_id,Input,Output,Vote Average,Vote Count,started_at,finished_at,Annotation,description,Output-Label
0,04f19b75-3909-477e-b6b5-c7f9aac0ba6c,Madres\n\nA Mexican-American couple expecting ...,Horror,6.2,58,1720139952,1720139968,good,A Mexican-American couple expecting their firs...,[Horror]
1,b2ca050a-d66d-4cdd-84fa-598066c91bc0,I fratelli De Filippo\n\nThe story of the De F...,"History, Drama",7.2,52,1719848817,1719848820,good,"The story of the De Filippo brothers, children...","[History, Drama]"
2,22f1bc6a-f40f-4951-b4f0-64675174b2c6,Puff: Wonders of the Reef\n\nA baby pufferfish...,Documentary,7.8,79,1720073582,1720073598,good,A baby pufferfish travels through a wondrous m...,[Documentary]
3,267eead3-7773-4db8-a9db-5c9b8aa97270,69 + 1\n\nA lesbian couple seeks the help of t...,Comedy,8.4,12,1720212057,1720212069,good,A lesbian couple seeks the help of their male ...,[Comedy]
4,23a62a79-4a39-4281-9369-960e23f1dbcf,Christmas at the Ranch\n\nWhen Haley Hollis re...,"Romance, Comedy",6.8,7,1720182317,1720182332,good,When Haley Hollis returns to her family ranch ...,"[Romance, Comedy]"


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
2,(History),(Drama),0.063,0.327,0.024,0.380952,1.164992,1.0,0.003399,1.087154,0.151147,0.065574,0.080167,0.227173
5,(Romance),(Comedy),0.137,0.231,0.050,0.364964,1.579929,1.0,0.018353,1.210954,0.425330,0.157233,0.174205,0.290707
6,(Romance),(Drama),0.137,0.327,0.049,0.357664,1.093774,1.0,0.004201,1.047739,0.099345,0.118072,0.045563,0.253756
13,(Fantasy),(Animation),0.083,0.095,0.025,0.301205,3.170577,1.0,0.017115,1.295086,0.746565,0.163399,0.227851,0.282181
36,(Animation),(Family),0.095,0.093,0.031,0.326316,3.508772,1.0,0.022165,1.346328,0.790055,0.197452,0.257239,0.329825
37,(Family),(Animation),0.093,0.095,0.031,0.333333,3.508772,1.0,0.022165,1.357500,0.788313,0.197452,0.263352,0.329825
39,(Family),(Comedy),0.093,0.231,0.029,0.311828,1.349905,1.0,0.007517,1.117453,0.285785,0.098305,0.105108,0.218685
40,"(Animation, Comedy)",(Family),0.021,0.093,0.012,0.571429,6.144393,1.0,0.010047,2.116333,0.855209,0.117647,0.527485,0.350230
41,"(Animation, Family)",(Comedy),0.031,0.231,0.012,0.387097,1.675744,1.0,0.004839,1.254684,0.416151,0.048000,0.202987,0.219522
42,"(Comedy, Family)",(Animation),0.029,0.095,0.012,0.413793,4.355717,1.0,0.009245,1.543824,0.793426,0.107143,0.352258,0.270054


Descriptions:


,description
0,A Mexican-American couple expecting their firs...
1,"The story of the De Filippo brothers, children..."
2,A baby pufferfish travels through a wondrous m...
3,A lesbian couple seeks the help of their male ...
4,When Haley Hollis returns to her family ranch ...



Binary Labels (y):


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [2]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
import torch.optim as optim

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
transformer_model = AutoModel.from_pretrained("distilbert-base-uncased").to(device)

# Define the PyTorch multi-label classifier model
class BaselineMovieClassifier(nn.Module):
    def __init__(self, transformer_model, num_labels):
        super(BaselineMovieClassifier, self).__init__()
        self.transformer = transformer_model
        self.classifier = nn.Linear(transformer_model.config.hidden_size, num_labels)

    def forward(self, input_ids):
        outputs = self.transformer(input_ids=input_ids)
        embeddings = outputs.last_hidden_state[:, 0, :]  # Get the [CLS] token embedding
        logits = self.classifier(embeddings)
        return logits

# Instantiate the model
num_genres = len(mlb.classes_)
baseline_model = BaselineMovieClassifier(transformer_model, num_genres).to(device)

# Define the loss function
criterion = torch.nn.BCEWithLogitsLoss()

# Define the optimizer
optimizer = optim.Adam(baseline_model.parameters(), lr=0.001)

# Tokenize the descriptions
X = tokenizer(
    text=df['description'].tolist(),
    add_special_tokens=True,
    max_length=128,
    truncation=True,
    padding='max_length',
    return_tensors='pt',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X['input_ids'], y, test_size=0.2, random_state=42)

# Create a dataloader for the training data
X_train_tensor = X_train.to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

# Training loop
epochs = 10
for epoch in range(epochs):
    baseline_model.train()
    total_loss = 0
    for batch_input_ids, batch_y_true in train_loader:
        optimizer.zero_grad()

        logits = baseline_model(batch_input_ids)
        loss = criterion(logits, batch_y_true)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_loader)}")

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch 1/10, Loss: 0.333955659866333
Epoch 2/10, Loss: 0.3218959230184555
Epoch 3/10, Loss: 0.3211858439445496
Epoch 4/10, Loss: 0.31953869879245755
Epoch 5/10, Loss: 0.32065181612968446
Epoch 6/10, Loss: 0.3942880457639694
Epoch 7/10, Loss: 0.32722590684890746
Epoch 8/10, Loss: 0.3209004634618759
Epoch 9/10, Loss: 0.3198554390668869
Epoch 10/10, Loss: 0.31911208391189577


In [3]:
from sklearn.metrics import classification_report

# Evaluation
baseline_model.eval()
with torch.no_grad():
    X_test_tensor = X_test.to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

    logits = baseline_model(X_test_tensor)
    y_pred = torch.sigmoid(logits)
    y_pred_binary = (y_pred > 0.5).cpu().numpy()

    print(classification_report(y_test, y_pred_binary, target_names=mlb.classes_, zero_division=0))

                 precision    recall  f1-score   support

         Action       0.00      0.00      0.00        19
      Adventure       0.00      0.00      0.00        18
      Animation       0.00      0.00      0.00        33
         Comedy       0.00      0.00      0.00        51
          Crime       0.00      0.00      0.00        18
    Documentary       0.00      0.00      0.00        28
          Drama       0.00      0.00      0.00        67
         Family       0.00      0.00      0.00        21
        Fantasy       0.00      0.00      0.00        25
        History       0.00      0.00      0.00         6
         Horror       0.00      0.00      0.00        19
          Music       0.00      0.00      0.00        17
        Mystery       0.00      0.00      0.00        10
        Romance       0.00      0.00      0.00        28
Science Fiction       0.00      0.00      0.00        11
       TV Movie       0.00      0.00      0.00         7
       Thriller       0.00    

In [4]:
# Load the evaluation data
eval_df = pd.read_csv('/content/drive/My Drive/movie-genre-prediction/evaluation_set.csv')

# Preprocess the evaluation data
eval_descriptions = eval_df['Input'].apply(lambda x: x.split('\n\n', 1)[1] if '\n' in x else '').tolist()
eval_X = tokenizer(
    text=eval_descriptions,
    add_special_tokens=True,
    max_length=128,
    truncation=True,
    padding='max_length',
    return_tensors='pt',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

# Make predictions on the evaluation data
baseline_model.eval()
with torch.no_grad():
    eval_X_tensor = eval_X['input_ids'].to(device)

    logits = baseline_model(eval_X_tensor)
    y_pred = torch.sigmoid(logits)
    predicted_labels_binary = (y_pred > 0.5).cpu().numpy()

# Convert the binary predictions to labels
predicted_labels = mlb.inverse_transform(predicted_labels_binary)

# Add the predicted labels to the evaluation dataframe
eval_df['predicted_genres_baseline'] = predicted_labels

# Transform the true and predicted labels using the same binarizer for a fair comparison
y_true_eval = mlb.transform(eval_df['expected_output'].str.split(', '))

# Generate the classification report
print("Classification Report for baseline model on the evaluation set:")
print(classification_report(y_true_eval, predicted_labels_binary, target_names=mlb.classes_, zero_division=0))

display(eval_df.head())

Classification Report for baseline model on the evaluation set:
                 precision    recall  f1-score   support

         Action       0.00      0.00      0.00        12
      Adventure       0.00      0.00      0.00         3
      Animation       0.00      0.00      0.00         5
         Comedy       0.00      0.00      0.00        36
          Crime       0.00      0.00      0.00         8
    Documentary       0.00      0.00      0.00        28
          Drama       0.00      0.00      0.00        49
         Family       0.00      0.00      0.00         8
        Fantasy       0.00      0.00      0.00         3
        History       0.00      0.00      0.00        10
         Horror       0.00      0.00      0.00        18
          Music       0.00      0.00      0.00         2
        Mystery       0.00      0.00      0.00         0
        Romance       0.00      0.00      0.00        17
Science Fiction       0.00      0.00      0.00         7
       TV Movie       0

,user_interaction_id,Input,Output,Vote Average,Vote Count,Annotation,expected_output,predicted_genres_baseline
0,e74dbc6c-36df-4822-b4df-913ae6c7a8bc,Spirit of a Denture\n\nDr. Middling is a denti...,"Adventure, Comedy",5.7,7,good,"Adventure, Comedy",()
1,f37f14a1-a013-4b01-88bc-9338c5a7c44a,We Ate the Children Last\n\nResearchers discov...,Comedy,6.2,6,good,Comedy,()
2,79edf183-8880-4141-b91a-475b429fc230,Castle Freak\n\nAfter she’s permanently blinde...,Horror,4.8,43,good,Horror,()
3,1e83532c-5a4c-40d9-a2cf-cc0d93d205a9,"My Man Is a Loser\n\nWhen it comes to women, p...",Comedy,4.5,29,good,Comedy,()
4,02d57491-c75e-4f1b-901b-64db45e9d78c,"Chirakodinja Kinavukal\n\nSumathi, a village g...",Comedy,7.2,11,good,Comedy,()


In [5]:
!pip install ltntorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [5]:
import torch
import torch.nn as nn
import ltn
from transformers import AutoTokenizer, AutoModel

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
transformer_model = AutoModel.from_pretrained("distilbert-base-uncased").to(device)

# Define the LTN model
class LTNMovieClassifier(nn.Module):
    def __init__(self, transformer_model, genres):
        super(LTNMovieClassifier, self).__init__()
        self.transformer = transformer_model
        self.predicates = nn.ModuleDict({
            genre: ltn.Predicate(
                nn.Sequential(
                    nn.Linear(transformer_model.config.hidden_size, 1),
                    nn.Sigmoid()
                )
            )
            for genre in genres
        })

    def forward(self, input_ids):
        outputs = self.transformer(input_ids=input_ids)
        embeddings = outputs.last_hidden_state[:, 0, :]
        return embeddings

# Instantiate the model
genres = mlb.classes_
ltn_model = LTNMovieClassifier(transformer_model, genres).to(device)

print("LTN Model Architecture:")
print(ltn_model)

LTN Model Architecture:
LTNMovieClassifier(
  (transformer): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace

In [8]:
import torch.optim as optim
from sklearn.model_selection import train_test_split
import ltn

# Define the fuzzy operators
And = ltn.Connective(ltn.fuzzy_ops.AndProd())
Or = ltn.Connective(ltn.fuzzy_ops.OrMax())
Implies = ltn.Connective(ltn.fuzzy_ops.ImpliesReichenbach())
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(), quantifier="f")

def ltn_loss(y_true, y_pred, axioms):
    bce_loss = torch.nn.functional.binary_cross_entropy(y_pred, y_true)
    axiom_loss = 0.0
    if axioms:
        axiom_values = torch.stack([axiom.value for axiom in axioms])
        axiom_loss = 1 - torch.mean(axiom_values)
    return bce_loss + axiom_loss

# Tokenize the descriptions
X = tokenizer(
    text=df['description'].tolist(),
    add_special_tokens=True,
    max_length=128,
    truncation=True,
    padding='max_length',
    return_tensors='pt',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X['input_ids'], y, test_size=0.2, random_state=42)

# Create a dataloader for the training data
X_train_tensor = X_train.to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

# Define the optimizer
optimizer = optim.Adam(ltn_model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    for batch_input_ids, batch_y_true in train_loader:
        optimizer.zero_grad()

        embeddings = ltn_model(batch_input_ids)
        x_embeddings = ltn.Variable("x_embeddings", embeddings)

        axioms = []
        for index, rule in high_confidence_rules.iterrows():
            antecedent = list(rule['antecedents'])[0]
            consequent = list(rule['consequents'])[0]

            if antecedent in ltn_model.predicates and consequent in ltn_model.predicates:
                axioms.append(Forall(x_embeddings,
                                     Implies(ltn_model.predicates[antecedent](x_embeddings),
                                             ltn_model.predicates[consequent](x_embeddings)),
                                     p=2))


        # Get predictions from predicates
        pred_list = []
        for genre in genres:
            pred_list.append(ltn_model.predicates[genre](x_embeddings).value.unsqueeze(1))
        batch_y_pred = torch.cat(pred_list, dim=1)

        # Calculate loss
        loss = ltn_loss(batch_y_true, batch_y_pred, axioms)

        loss.backward(retain_graph=True)
        optimizer.step()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

Epoch 1/10, Loss: 0.31696441769599915
Epoch 2/10, Loss: 0.29443359375
Epoch 3/10, Loss: 0.35017362236976624
Epoch 4/10, Loss: 0.3833513855934143
Epoch 5/10, Loss: 0.44842132925987244
Epoch 6/10, Loss: 0.32091739773750305
Epoch 7/10, Loss: 0.5067225694656372
Epoch 8/10, Loss: 0.28009018301963806
Epoch 9/10, Loss: 0.30389830470085144
Epoch 10/10, Loss: 0.3647962510585785


In [9]:
from sklearn.metrics import classification_report

# Load the evaluation data
eval_df = pd.read_csv('/content/drive/My Drive/movie-genre-prediction/evaluation_set.csv')

# Preprocess the evaluation data
eval_descriptions = eval_df['Input'].apply(lambda x: x.split('\n\n', 1)[1] if '\n' in x else '').tolist()
eval_X = tokenizer(
    text=eval_descriptions,
    add_special_tokens=True,
    max_length=128,
    truncation=True,
    padding='max_length',
    return_tensors='pt',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

# Make predictions on the evaluation data
ltn_model.eval()
with torch.no_grad():
    eval_X_tensor = eval_X['input_ids'].to(device)

    embeddings = ltn_model(eval_X_tensor)
    x_embeddings = ltn.Variable("x_embeddings", embeddings)

    y_pred_list = [ltn_model.predicates[genre](x_embeddings).value.unsqueeze(1) for genre in genres]
    y_pred = torch.cat(y_pred_list, dim=1)

    predicted_labels_binary = (y_pred > 0.5).cpu().numpy()

# Convert the binary predictions to labels
predicted_labels = mlb.inverse_transform(predicted_labels_binary)

# Add the predicted labels to the evaluation dataframe
eval_df['predicted_genres_ltn'] = predicted_labels

# Transform the true and predicted labels using the same binarizer for a fair comparison
y_true_eval = mlb.transform(eval_df['expected_output'].str.split(', '))

# Generate the classification report
print("Classification Report for LTN-enhanced model on the evaluation set:")
print(classification_report(y_true_eval, predicted_labels_binary, target_names=genres, zero_division=0))

display(eval_df.head())

Classification Report for LTN-enhanced model on the evaluation set:
                 precision    recall  f1-score   support

         Action       0.00      0.00      0.00        12
      Adventure       0.00      0.00      0.00         3
      Animation       0.00      0.00      0.00         5
         Comedy       0.00      0.00      0.00        36
          Crime       0.00      0.00      0.00         8
    Documentary       0.00      0.00      0.00        28
          Drama       0.00      0.00      0.00        49
         Family       0.00      0.00      0.00         8
        Fantasy       0.00      0.00      0.00         3
        History       0.00      0.00      0.00        10
         Horror       0.00      0.00      0.00        18
          Music       0.00      0.00      0.00         2
        Mystery       0.00      0.00      0.00         0
        Romance       0.00      0.00      0.00        17
Science Fiction       0.00      0.00      0.00         7
       TV Movie    

,user_interaction_id,Input,Output,Vote Average,Vote Count,Annotation,expected_output,predicted_genres_ltn
0,e74dbc6c-36df-4822-b4df-913ae6c7a8bc,Spirit of a Denture\n\nDr. Middling is a denti...,"Adventure, Comedy",5.7,7,good,"Adventure, Comedy",()
1,f37f14a1-a013-4b01-88bc-9338c5a7c44a,We Ate the Children Last\n\nResearchers discov...,Comedy,6.2,6,good,Comedy,()
2,79edf183-8880-4141-b91a-475b429fc230,Castle Freak\n\nAfter she’s permanently blinde...,Horror,4.8,43,good,Horror,()
3,1e83532c-5a4c-40d9-a2cf-cc0d93d205a9,"My Man Is a Loser\n\nWhen it comes to women, p...",Comedy,4.5,29,good,Comedy,()
4,02d57491-c75e-4f1b-901b-64db45e9d78c,"Chirakodinja Kinavukal\n\nSumathi, a village g...",Comedy,7.2,11,good,Comedy,()


In [ ]:
## Model Performance Comparison

### Baseline Model:

The baseline model, a standard multi-label classifier using a pre-trained DistilBERT model, performs poorly on the evaluation set. The classification report shows precision, recall, and F1-scores of 0.00 for all genres. This indicates that the model fails to correctly predict any of the genres in the evaluation data. The `predicted_genres_baseline` column in the `eval_df` DataFrame is empty for all samples, confirming that the model did not make any positive predictions.

### LTN-enhanced Model:

Similarly, the LTN-enhanced model, which incorporates logical axioms derived from association rule mining, also shows no improvement in performance. The classification report for this model is identical to the baseline model, with all metrics at 0.00. The `predicted_genres_ltn` column is also empty, indicating a failure to predict any genres.

### Conclusion:

Both the baseline and the LTN-enhanced models completely fail to generalize to the evaluation set. Several factors could contribute to this poor performance:

- **Data Quality:** The descriptions might not contain enough information to distinguish between genres.
- **Model Complexity:** The models might be too complex for the given data, leading to overfitting on the training set.
- **Hyperparameter Tuning:** The learning rate, batch size, and number of epochs might not be optimal.
- **Axiom Quality:** The association rules used as axioms in the LTN model might not be strong enough or might not generalize well to unseen data.

Further investigation is needed to diagnose the root cause of the issue. This could involve:

- **Error Analysis:** Manually inspecting the model's predictions on the training set to understand where it is failing.
- **Data Augmentation:** Increasing the size and diversity of the training data.
- **Hyperparameter Optimization:** Systematically tuning the model's hyperparameters.
- **Feature Engineering:** Exploring different ways to represent the input text.